# LIVRIA : multilabel classification


Ce notebook comprend le code implementant notre modèle de régression linéaire pour la prédiction des thèmes susceptibles de plaire à l'utilisateur en fonction des critères d'entrée qu'il aura renseignés. Je précise que cela ne compose qu'une partie des prédictions? En effet, ce modèle sera complété par du filtrage collaboratif pour les thèmes mais aussi pour les livres du set de données de Goodbooks-10k.

### Import des librairies et des données 

In [1]:
# Librairies
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
%matplotlib inline
# On configure les avertissements qu'on souhaite ignorer
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Données
dataCrit = pd.read_csv('data/df_entree.csv', sep='\t')
dataTheme = pd.read_csv('data/df_sortie.csv', sep='\t')
# On supprime la colonne inutile :
del dataTheme['Unnamed: 0']
del dataCrit['Unnamed: 0']

In [2]:
dataCrit.head(10)

,Agite,Altruiste,Ambitieux,Amusant,Autoritaire,Aventurier,Calme,Connaissance,Consciencieux,Creatif,...,Reserve,Rien faire,Sexe,Sociable,Sport,Sportif,Style,Theatre,Tout,Voyage
0,0,0,0,0,0,0,1,0,1,1,...,0,0,1,0,0,0,0,0,1,1
1,0,1,0,0,0,1,1,0,1,0,...,0,1,1,0,1,0,0,0,0,1
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,1,1,0,0,0,1
3,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,1,0,0,0,1
4,0,0,0,1,0,0,0,1,0,0,...,0,0,1,1,1,1,0,0,0,0
5,0,0,1,0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,1
6,1,0,0,1,0,0,0,1,1,1,...,0,0,0,1,1,0,0,0,0,1
7,0,0,1,1,0,1,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0
8,0,1,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,1,0,0,1
9,0,1,0,0,0,0,0,1,1,0,...,0,1,0,1,1,0,1,1,0,0


In [3]:
dataTheme.head(10)

,ArtsCulture,BdComics,DocMedia,Erotisme,Esoterisme,HistGeo,Jeunesse,LittEtrangere,LoisirVie,Philosophie,RomanFiction,SHS,SanteBE,ScienceTechnique
0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,1,0,0,1,0,0,0
2,0,0,0,0,0,0,1,1,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,1,1,0,0,1,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,1,1,0,0
6,1,0,1,1,0,0,0,1,0,1,1,0,0,0
7,0,0,0,0,1,0,0,0,0,1,0,1,0,0
8,0,0,0,0,0,0,0,0,0,0,1,0,0,0
9,0,1,0,0,0,1,0,1,0,0,1,1,0,0


On créer un set qui réunit toutes les données

In [4]:
df_entier = pd.concat([dataCrit,dataTheme], axis=1)
df_entier.head(10)

,Agite,Altruiste,Ambitieux,Amusant,Autoritaire,Aventurier,Calme,Connaissance,Consciencieux,Creatif,...,Esoterisme,HistGeo,Jeunesse,LittEtrangere,LoisirVie,Philosophie,RomanFiction,SHS,SanteBE,ScienceTechnique
0,0,0,0,0,0,0,1,0,1,1,...,0,0,0,0,0,0,1,0,0,0
1,0,1,0,0,0,1,1,0,1,0,...,0,0,0,1,0,0,1,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,1,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
5,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,1,0,0
6,1,0,0,1,0,0,0,1,1,1,...,0,0,0,1,0,1,1,0,0,0
7,0,0,1,1,0,1,1,0,0,1,...,1,0,0,0,0,1,0,1,0,0
8,0,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
9,0,1,0,0,0,0,0,1,1,0,...,0,1,0,1,0,0,1,1,0,0


In [5]:
# On vérifie qu'il n'y a pas de valeur manquante dans notre dataFrame
df_entier.isnull().sum()

Agite               0
Altruiste           0
Ambitieux           0
Amusant             0
Autoritaire         0
Aventurier          0
Calme               0
Connaissance        0
Consciencieux       0
Creatif             0
Cuisine             0
Curieux             0
Dessin              0
Esprit              0
FacileLire          0
Geek                0
Intellectuel        0
Introverti          0
Jaloux              0
Jeux videos         0
Meditation          0
Pantouflard         0
Personnage          0
Reflechir           0
Reserve             0
Rien faire          0
Sexe                0
Sociable            0
Sport               0
Sportif             0
Style               0
Theatre             0
Tout                0
Voyage              0
ArtsCulture         0
BdComics            0
DocMedia            0
Erotisme            0
Esoterisme          0
HistGeo             0
Jeunesse            0
LittEtrangere       0
LoisirVie           0
Philosophie         0
RomanFiction        0
SHS       

Pas de NaN values dans nos sets donc nous pouvons continuer.

## Entrainement du modèle

On créer les sets de test et d'entrainement

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
train, test = train_test_split(df_entier, test_size=0.30, shuffle=True)

x_train = train.drop(['ArtsCulture','BdComics','DocMedia','Erotisme','HistGeo','HistGeo','Jeunesse','LittEtrangere','LoisirVie','Philosophie','RomanFiction','SHS','SanteBE','ScienceTechnique'],axis=1)
y_train = train.drop(['Agite','Altruiste','Ambitieux','Amusant','Autoritaire','Aventurier','Calme','Connaissance','Consciencieux','Creatif','Cuisine','Curieux','Dessin','Esprit','FacileLire','Geek','Intellectuel','Introverti','Jaloux','Jeux videos','Meditation','Pantouflard','Personnage','Reflechir','Reserve','Rien faire','Sexe','Sociable','Sport','Sportif','Style','Theatre','Tout','Voyage'], axis=1)
x_test = test.drop(['ArtsCulture','BdComics','DocMedia','Erotisme','HistGeo','HistGeo','Jeunesse','LittEtrangere','LoisirVie','Philosophie','RomanFiction','SHS','SanteBE','ScienceTechnique'],axis=1)
y_test = test.drop(['Agite','Altruiste','Ambitieux','Amusant','Autoritaire','Aventurier','Calme','Connaissance','Consciencieux','Creatif','Cuisine','Curieux','Dessin','Esprit','FacileLire','Geek','Intellectuel','Introverti','Jaloux','Jeux videos','Meditation','Pantouflard','Personnage','Reflechir','Reserve','Rien faire','Sexe','Sociable','Sport','Sportif','Style','Theatre','Tout','Voyage'], axis=1)

## Regression logistique

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier

In [26]:
# On utilise un pipeline pour utiliser la régression logistique sur ce problème de classification multitache
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])
themes = list(dataTheme.columns.values)

for theme in themes :
    print('**Calculs des prédictions pour {}...**'.format(theme))
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(x_train, train[theme])
    
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(x_test)
    #print(prediction)
    print('Taux de réussite : {}'.format(accuracy_score(test[theme], prediction)))
    print("\n")

**Calculs des prédictions pour ArtsCulture...**
Taux de réussite : 0.8854166666666666


**Calculs des prédictions pour BdComics...**
Taux de réussite : 0.7526041666666666


**Calculs des prédictions pour DocMedia...**
Taux de réussite : 0.8697916666666666


**Calculs des prédictions pour Erotisme...**
Taux de réussite : 0.8984375


**Calculs des prédictions pour Esoterisme...**
Taux de réussite : 1.0


**Calculs des prédictions pour HistGeo...**
Taux de réussite : 0.7916666666666666


**Calculs des prédictions pour Jeunesse...**
Taux de réussite : 0.7916666666666666


**Calculs des prédictions pour LittEtrangere...**
Taux de réussite : 0.6328125


**Calculs des prédictions pour LoisirVie...**
Taux de réussite : 0.90625


**Calculs des prédictions pour Philosophie...**
Taux de réussite : 0.8177083333333334


**Calculs des prédictions pour RomanFiction...**
Taux de réussite : 0.9192708333333334


**Calculs des prédictions pour SHS...**
Taux de réussite : 0.75


**Calculs des prédictions 

On obtient de très bons résultats pour la prédiction. Voyons voir si d'autres modèles pourraient être plus performants

## Classification naïve bayésienne

In [10]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

In [11]:
# On utilise la classification naïve bayésienne (forte indépendance des hypothèses)
classifier = BinaryRelevance(GaussianNB())

In [12]:
# Entraînement
classifier.fit(x_train, y_train)
# Predictions
predictions = classifier.predict(x_test)
# Performance
print("Taux de réussite = ",accuracy_score(y_test,predictions))

Taux de réussite =  0.1015625


Prédictions en dessous de 10%, c'est bien inférieure aux résultats obtenus avec la régression logistique.

## Méthode dite "Classifier Chains"

In [13]:
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression

In [14]:
# On initialise le classificateur multi-label
classifier = ClassifierChain(LogisticRegression())

In [15]:
# On entraîne le modèle de régression logistique sur les données d'entraînement
classifier.fit(x_train, y_train)

ClassifierChain(classifier=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
        order=None, require_dense=[True, True])

In [16]:
# predictions
predictions = classifier.predict(x_test)

In [17]:
# accuracy
print("Taux de réussite = ",accuracy_score(y_test,predictions))

Taux de réussite =  0.15625


Performance toujours trop basse par rapport aux prédictions qu'on parvient à faire avec la méthode de regression logistique.

## Méthode dite Label powerset

In [18]:
from skmultilearn.problem_transform import LabelPowerset

In [19]:
# On initialise le classificateur
classifier = LabelPowerset(LogisticRegression())

In [20]:
# On entraine le modèle
classifier.fit(x_train, y_train)

LabelPowerset(classifier=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       require_dense=[True, True])

In [21]:
# predictions
predictions = classifier.predict(x_test)

In [22]:
# performance
print("Taux de réussite = ",accuracy_score(y_test,predictions))

Taux de réussite =  0.11197916666666667


Bien que cette méthode soit en général très puissance, ses performances ne semblent pas être à la hauteur par rapport à la première méthode. Essayons une dernière méthode avant de faire notre choix définitivement.

## Méthode des k plus proches voisins

Cette méthode n'a pas de phase d'apprentissage contraîrement aux méthodes utilisées précédemment. Elle se base simplement sur le jeux de données pour faire ses prédictions. Son fonctionnement peut être assimilé à l’analogie suivante : “dis moi qui sont tes voisins, je te dirais qui tu es…”.

In [23]:
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix

In [24]:
x_train = lil_matrix(x_train).toarray()
y_train = lil_matrix(y_train).toarray()
x_test = lil_matrix(x_test).toarray()

In [25]:
k=np.arange(1,15)
for k in k :
    print('**Prédictions avec les {} voisins...**'.format(k))
    # train
    classifier_new = MLkNN(k=k)
    classifier_new.fit(x_train, y_train)
    # predict
    predictions_new = classifier_new.predict(x_test)
    # accuracy
    print('Taux de réussite : ' + str(accuracy_score(y_test,predictions_new)) + '\n')

**Prédictions avec les 1 voisins...**
Taux de réussite : 0.049479166666666664

**Prédictions avec les 2 voisins...**
Taux de réussite : 0.03125

**Prédictions avec les 3 voisins...**
Taux de réussite : 0.0546875

**Prédictions avec les 4 voisins...**
Taux de réussite : 0.052083333333333336

**Prédictions avec les 5 voisins...**
Taux de réussite : 0.08333333333333333

**Prédictions avec les 6 voisins...**
Taux de réussite : 0.109375

**Prédictions avec les 7 voisins...**
Taux de réussite : 0.1015625

**Prédictions avec les 8 voisins...**
Taux de réussite : 0.09375

**Prédictions avec les 9 voisins...**
Taux de réussite : 0.109375

**Prédictions avec les 10 voisins...**
Taux de réussite : 0.09375

**Prédictions avec les 11 voisins...**
Taux de réussite : 0.11458333333333333

**Prédictions avec les 12 voisins...**
Taux de réussite : 0.109375

**Prédictions avec les 13 voisins...**
Taux de réussite : 0.12239583333333333

**Prédictions avec les 14 voisins...**
Taux de réussite : 0.1171875



En conclusion, c'est le premier modèle qui semble donner les meilleurs résultats pour la prédiction de thèmes. Cette méthode transforme ce problèmede classification multi-label en des problèmes plus simple de classification binaire protée sur chaque thème indépendamment les uns des autres. Cela rend la prédiction plus pertinente mais nous devons préciser que le lien entre les thèmes est totalement ignoré. C'est pourquoi le filtrage collaborratif que nous réalisons dans un notebook à part est pertinent pour compléter cette approche et suggérer des thèmes en fonction de ceux qu'aiment l'utilisateur ou en se bsant sur la similarité entre utilisateur (cf. Notebook Themes_Collaborative_filtering).

Je vous invite maintenant à aller voir la mise en place de notre modèle dont nous présentons les prédictions dans le notebook Livria_recommender_system.